In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix


ModuleNotFoundError: No module named 'pandas'

In [5]:
# Load dataset
data = pd.read_csv('data_finish.csv', delimiter = ',')

# Drop unnecessary columns
data.drop(['ddd_car', 'station_id', 'station_name', 'region_name', 'date'], axis=1, inplace=True)

# Display the first few rows of the dataset
print(data.head())

     Tn    Tx  Tavg  RH_avg    RR   ss  ff_x  ddd_x  ff_avg  flood
0  26.0  34.8  28.6    81.0   NaN  5.8   5.0  280.0     2.0      0
1  25.6  33.2  27.0    88.0   1.6  8.7   4.0  290.0     2.0      1
2  24.4  34.9  28.1    80.0  33.8  5.4   4.0  280.0     2.0      1
3  24.8  33.6  29.2    81.0   NaN  6.6   3.0  200.0     1.0      0
4  25.8  33.6  26.7    91.0   NaN  3.2   3.0  180.0     1.0      0


In [6]:
# Menggantikan nilai NaN dengan rata-rata kolom
data.fillna(data.mean(), inplace=True)

print(data.isnull().sum())

Tn        0
Tx        0
Tavg      0
RH_avg    0
RR        0
ss        0
ff_x      0
ddd_x     0
ff_avg    0
flood     0
dtype: int64


In [7]:
X = data[['Tn', 'Tx', 'Tavg', 'RH_avg', 'RR', 'ss', 'ff_x']]
y = data['flood']

In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [11]:
# Menyusun model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
# Melatih model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

Epoch 1/50
64/64 [==============================] - 1s 16ms/step - loss: 0.4710 - accuracy: 0.8621 - val_loss: 0.2303 - val_accuracy: 0.9257
Epoch 2/50
64/64 [==============================] - 0s 2ms/step - loss: 0.2663 - accuracy: 0.9203 - val_loss: 0.2108 - val_accuracy: 0.9257
Epoch 3/50
64/64 [==============================] - 0s 2ms/step - loss: 0.2273 - accuracy: 0.9307 - val_loss: 0.2073 - val_accuracy: 0.9267
Epoch 4/50
64/64 [==============================] - 0s 2ms/step - loss: 0.2356 - accuracy: 0.9272 - val_loss: 0.2066 - val_accuracy: 0.9267
Epoch 5/50
64/64 [==============================] - 0s 2ms/step - loss: 0.2175 - accuracy: 0.9379 - val_loss: 0.2061 - val_accuracy: 0.9277
Epoch 6/50
64/64 [==============================] - 0s 2ms/step - loss: 0.2123 - accuracy: 0.9289 - val_loss: 0.2066 - val_accuracy: 0.9277
Epoch 7/50
64/64 [==============================] - 0s 2ms/step - loss: 0.1986 - accuracy: 0.9337 - val_loss: 0.2079 - val_accuracy: 0.9277
Epoch 8/50
64/64 [=

In [13]:
# Evaluasi model pada data uji
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')

40/40 [==============================] - 0s 564us/step - loss: 0.2557 - accuracy: 0.9120
Test Accuracy: 0.91204434633255


In [14]:
# Membuat prediksi pada data uji
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

In [15]:
# Menampilkan confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Menampilkan laporan klasifikasi
print("Classification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[1149    5]
 [ 106    2]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.95      1154
           1       0.29      0.02      0.03       108

    accuracy                           0.91      1262
   macro avg       0.60      0.51      0.49      1262
weighted avg       0.86      0.91      0.88      1262



In [16]:
# Simpan model setelah pelatihan
model.save('flood_prediction_model.h5')

In [18]:
import joblib

# Simpan scaler
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']